<a href="https://colab.research.google.com/github/robert-myers/myanimelist-recommender/blob/master/notebooks/try_it_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install jikanpy && pip install surprise

     |████████████████████████████████| 1.2MB 5.5MB/s 
     |████████████████████████████████| 256kB 58.7MB/s 
     |████████████████████████████████| 153kB 54.5MB/s 
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl size=3162 sha256=2f64acd73c7fe36329978d1bc6778465dcc2431da20a3d30e9632053bccd3793
  Stored in directory: /root/.cache/pip/wheels/d3/00/b3/32d613e19e08a739751dd6bf998cfed277728f8b2127ad4eb7
Successfully built idna-ssl
     |████████████████████████████████| 6.5MB 2.9MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.0-cp36-cp36m-linux_x86_64.whl size=1678215 sha256=8b6358966ce684b47e147c1c5bca2cdad5f9d03352c4ccf0bf59d19b75eefd34
  Stored in directory: /root/.cache/pip/wheels/cc/fa/8c/16c93fccce688ae1bde7d979ff102f7bee980d9cfeb8641bcf
Successfully built scikit-surprise


In [0]:
import pandas as pd
import string
import time

from jikanpy import Jikan
from surprise import accuracy
from surprise.dump import load

In [0]:
jikan = Jikan()

In [0]:
users = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/UserList.csv")
users = users[["username"]]
users = users.dropna()
users["lower"] = users["username"].apply(lambda x: x.lower())
users = users.set_index("lower")

In [0]:
users = dict(users.T)

In [0]:
def get_uid(username):
  username = username.lower()
  if username in users:
    return users[username.lower()][0]
  else:
    return username

In [7]:
anime = pd.read_csv("https://s3.us-east-2.amazonaws.com/my.anime.list.sagemaker/myanimelist/AnimeList.csv", index_col="anime_id")[["title",	"image_url"]]
anime.reset_index()[["anime_id"]]

,anime_id
0,11013
1,2104
2,5262
3,721
4,12365
...,...
14473,26089
14474,21525
14475,37897
14476,34193


In [0]:
surprised = load("/content/drive/My Drive/svd.pickle")

In [0]:
algo = surprised[1]
predictions = surprised[0]

In [10]:
accuracy.rmse(predictions)

RMSE: 1.1859


1.1859182533000143

In [11]:
"RX-78-2_Gundam".translate(str.maketrans("", "", string.punctuation))

'RX782Gundam'

In [12]:
anime.reset_index()["anime_id"]

0        11013
1         2104
2         5262
3          721
4        12365
         ...  
14473    26089
14474    21525
14475    37897
14476    34193
14477    37908
Name: anime_id, Length: 14478, dtype: int64

In [0]:
def get_recommendations(user, num_titles=10, filter_completed=True):
  user = get_uid(user)  
  user_predictions = anime.reset_index()[["anime_id"]]
  if filter_completed:
    user_completed = pd.DataFrame(jikan.user(username=user, request='animelist', argument='completed', page=1)["anime"])
    i = 2
    downloading = True
    while downloading:
      time.sleep(4) # API requires 4 second delay between requests
      downloading = jikan.user(username=user, request='animelist', argument='completed', page=i)["anime"]
      user_completed = user_completed.append(pd.DataFrame(downloading))
      i += 1
    user_completed_lst = list(user_completed["mal_id"])
    user_predictions["est"] = user_predictions["anime_id"].apply(lambda item: algo.predict(user, item)[3] if item not in user_completed_lst else 0)
  else:
    user_predictions["est"] = user_predictions["anime_id"].apply(lambda item: algo.predict(user, item)[3])
  return user_predictions.sort_values("est", ascending=False).head(10)["anime_id"].apply(lambda x: anime.loc[x])

In [20]:
%%time
get_recommendations("orionite", filter_completed=True)

CPU times: user 270 ms, sys: 4.07 ms, total: 274 ms
Wall time: 9.12 s


,title,image_url
7483,Redline,https://myanimelist.cdn-dena.com/images/anime/...
4687,Detroit Metal City,https://myanimelist.cdn-dena.com/images/anime/...
3677,Prison School,https://myanimelist.cdn-dena.com/images/anime/...
10598,Daicon Opening Animations,https://myanimelist.cdn-dena.com/images/anime/...
10810,JoJo no Kimyou na Bouken (TV),https://myanimelist.cdn-dena.com/images/anime/...
13232,Nihon Animator Mihonichi,https://myanimelist.cdn-dena.com/images/anime/...
9889,Great Teacher Onizuka,https://myanimelist.cdn-dena.com/images/anime/...
6687,Mahou Shoujo Lyrical Nanoha: The Movie 2nd A&#...,https://myanimelist.cdn-dena.com/images/anime/...
5005,Hellsing Ultimate,https://myanimelist.cdn-dena.com/images/anime/...
9880,Uchuu Senkan Yamato 2199,https://myanimelist.cdn-dena.com/images/anime/...


In [21]:
%%time
get_recommendations("orionite", filter_completed=False)

CPU times: user 148 ms, sys: 11.1 ms, total: 159 ms
Wall time: 150 ms


,title,image_url
6040,FLCL,https://myanimelist.cdn-dena.com/images/anime/...
5257,Suzumiya Haruhi no Yuuutsu,https://myanimelist.cdn-dena.com/images/anime/...
11361,Byousoku 5 Centimeter,https://myanimelist.cdn-dena.com/images/anime/...
10918,Tengen Toppa Gurren Lagann,https://myanimelist.cdn-dena.com/images/anime/...
13587,Carnival Phantasm,https://myanimelist.cdn-dena.com/images/anime/...
12938,Tengen Toppa Gurren Lagann Movie 2: Lagann-hen,https://myanimelist.cdn-dena.com/images/anime/...
1094,Suzumiya Haruhi no Shoushitsu,https://myanimelist.cdn-dena.com/images/anime/...
9140,Mahou Shoujo Madoka★Magica,https://myanimelist.cdn-dena.com/images/anime/...
1389,Evangelion: 2.0 You Can (Not) Advance,https://myanimelist.cdn-dena.com/images/anime/...
6112,Golden Boy,https://myanimelist.cdn-dena.com/images/anime/...
